In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import random

In [ ]:
## libraries to create the model 
from tensorflow.keras.layers import  MaxPool1D, Input, MaxPool1D, UpSampling1D
from tensorflow.keras.layers import  Dense, Activation, Dropout, Reshape, Conv1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Model
import tensorflow.keras.backend as K

In [ ]:
# For reproducibility
np.random.seed(12)
tf.random.set_seed(12)
random.seed(1)

In [ ]:
dir_saved_data = "/content/drive/My Drive/ASHRAEData/"
dir_save_model = "/content/drive/My Drive/ASHRAEData/" 

In [ ]:
## CAN BE CHANGED 

## change these lists as per model's input 
site_id = 0 

## window length 
seq_length = 24 

## name of the model 
model_name = 'Baseline_Conv1'

BATCH_SIZE_TRAIN = 32 ## Training batch size
BATCH_SIZE_VAL = 16 ## Validation batch size

EPOCHS = 500

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
def generate_model_autoencoder1(input_len, name):
  ## Creating the Non Conditional Convolutional De-noising Autoencoder Model

  ## input layer
  input_encoder = Input(shape=(input_len,)) 
  ## introducing Noise (can be changed)
  ip = Dropout(0.05)(input_encoder) 
  encoder = Reshape((input_len,1))(ip)

  ## First layer of the encoder 
  encoder = Conv1D(12, 2, strides=1, padding='same', data_format='channels_last', activation='tanh')(encoder)
  encoder = MaxPool1D(pool_size=2, data_format='channels_last')(encoder)

  ## Second layer of the encoder 
  encoder = Conv1D(6, 2, strides=1, padding='same', data_format='channels_last', activation='tanh')(encoder)  
  encoder = MaxPool1D(pool_size=2, data_format='channels_last')(encoder)

  ## Third and last layer of the encoder 
  encoder = Conv1D(1, 2, strides=1, padding='same', data_format='channels_last', activation='tanh')(encoder)  
  encoder = MaxPool1D(pool_size=2, data_format='channels_last')(encoder)

  ## Decoder first layer
  decoder = Conv1D(1, 2, strides=1, padding='same', data_format='channels_last', activation='tanh')(encoder)  
  decoder = UpSampling1D(size=2)(decoder)

  ## Decoder second layer
  decoder = Conv1D(6, 2, strides=1, padding='same', data_format='channels_last', activation='tanh')(decoder)  
  decoder = UpSampling1D(size=2)(decoder)

  ## Decoder third layer
  decoder = Conv1D(12, 2, strides=1, padding='same', data_format='channels_last', activation='tanh')(decoder)  
  decoder = UpSampling1D(size=2)(decoder)

  ## output layer
  decoded = decoder = Conv1D(1, 2, strides=1, padding='same', data_format='channels_last', activation='relu')(decoder) 
  
  ## initalizing keras.Model instance
  model = Model(input_encoder, decoded, name = name)

  return model

In [ ]:
def compile_model(model):
  ## function to compile the model 

  opt = Adam(lr=0.0001)
  model.compile(optimizer=opt, loss='mse', metrics='mse')
  return model

In [ ]:
## Creating the Baseline Model
autoencoder1 = generate_model_autoencoder1(seq_length,  model_name)

In [ ]:
## Compiling the Model
autoencoder1 = compile_model(autoencoder1)

In [ ]:
## Reading saved data
os.chdir(dir_saved_data)

## Reading training data
with open("./Baseline_data/Not_conditional/"+ "site_id_" + str(site_id)+"/non_anom/train_data.pkl", 'rb') as f:
  X_train_non_anom = pickle.load(f)

## Reading validation data
with open("./Baseline_data/Not_conditional/"+ "site_id_" + str(site_id)+"/non_anom/val_data.pkl", 'rb') as f:
  X_val_non_anom = pickle.load(f)

# ## Reading test data
# with open("./Baseline_data/Not_conditional/"+ "site_id_" + str(site_id)+"/non_anom/test_data.pkl", 'rb') as f:
#   X_test_non_anom = pickle.load(f)

In [ ]:
## creating tf.Dataset object required for training 
train_dataset_autoencoder1 = tf.data.Dataset.from_tensor_slices((X_train_non_anom,X_train_non_anom))

## creating tf.Dataset object required for validation
val_dataset_autoencoder1 = tf.data.Dataset.from_tensor_slices((X_val_non_anom, X_val_non_anom))

## Setting Batch size
train_dataset_autoencoder1 = train_dataset_autoencoder1.batch(BATCH_SIZE_TRAIN)
val_dataset_autoencoder1 = val_dataset_autoencoder1.batch(BATCH_SIZE_VAL)

print(train_dataset_autoencoder1)
print(val_dataset_autoencoder1)

<BatchDataset shapes: ((None, 24, 1), (None, 24, 1)), types: (tf.float32, tf.float32)>
<BatchDataset shapes: ((None, 24, 1), (None, 24, 1)), types: (tf.float32, tf.float32)>


In [ ]:
## Creating call-backs
os.chdir(dir_save_model)

if not os.path.exists(model_name + "/"):
  os.mkdir(model_name + "/")

checkpoint_path = './'+ model_name + "/" + "_site_id_"+ str(site_id) + '.ckpt'
model_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, monitor='val_loss', save_best_only=True,
    save_weights_only=True, mode='min', save_freq='epoch')

## tensorboard call-back
log_dir = 'logs/fit/'+model_name + "_site_id_"+ str(site_id)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

## Early Stopping call-back
early_stopping = tf.keras.callbacks.EarlyStopping( monitor='val_loss', patience=60, min_delta=0.0001)

In [ ]:
os.chdir(dir_save_model)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

In [ ]:
## Train the model
autoencoder1.fit(train_dataset_autoencoder1, epochs=EPOCHS, validation_data = val_dataset_autoencoder1, callbacks=[model_callback, tensorboard_callback, early_stopping])

Epoch 1/300
1155/1155 [==============================] - 7s 6ms/step - loss: 9.7583 - mse: 9.7583 - val_loss: 3.3464 - val_mse: 3.3464
Epoch 2/300
1155/1155 [==============================] - 6s 5ms/step - loss: 2.7799 - mse: 2.7799 - val_loss: 2.3471 - val_mse: 2.3471
Epoch 3/300
1155/1155 [==============================] - 7s 6ms/step - loss: 1.2832 - mse: 1.2832 - val_loss: 0.9571 - val_mse: 0.9571
Epoch 4/300
1155/1155 [==============================] - 7s 6ms/step - loss: 0.7819 - mse: 0.7819 - val_loss: 0.6665 - val_mse: 0.6665
Epoch 5/300
1155/1155 [==============================] - 7s 6ms/step - loss: 0.5280 - mse: 0.5280 - val_loss: 0.4532 - val_mse: 0.4532
Epoch 6/300
1155/1155 [==============================] - 7s 6ms/step - loss: 0.3510 - mse: 0.3510 - val_loss: 0.3148 - val_mse: 0.3148
Epoch 7/300
1155/1155 [==============================] - 7s 6ms/step - loss: 0.2383 - mse: 0.2383 - val_loss: 0.2330 - val_mse: 0.2330
Epoch 8/300
1155/1155 [==============================] 